In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_csv('../spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [2]:
def transform_data(df):
    cv = CountVectorizer()
    X = cv.fit_transform(df.Message)
    y = df.Category.apply(lambda x: 1 if x == "spam" else 0)
    return X, y

In [14]:
def calculate_prior(y):
    n_spam = np.sum(y == 1)
    n_ham = np.sum(y == 0)
    p_spam = n_spam / (n_spam + n_ham)
    p_ham = 1 - p_spam
    return p_spam, p_ham

In [6]:
def calculate_likelihood(X, y, p_spam, p_ham):
    a = X[np.array(y == 1)]
    b = X[np.array(y == 0)]
    n_spam = np.sum(a)
    n_ham = np.sum(b)
    v = X.shape[1]
    likelihood_spam = (np.sum(a, axis = 0) + 1) / (n_spam + v)
    likelihood_ham = (np.sum(b, axis = 0) + 1) / (n_ham + v)
    return likelihood_spam, likelihood_ham 

In [12]:
def predict(X, likelihood_spam, likelihood_ham, p_spam, p_ham):
    predicts = np.zeros(X.shape[0])
    x = 0
    for vector in X:
        spam = p_spam
        ham = p_ham
        for a in range(vector.shape[0]):
            if (vector[:, a] != 0):
                spam = spam * likelihood_spam[:, a]
                ham = ham * likelihood_ham[:, a]
            else:
                spam = spam * (1 - likelihood_spam[:, a])
                ham = ham * (1 - likelihood_ham[:, a])
        if (spam > ham):
            predicts[x] = 1
        x = x + 1
    return predicts

In [9]:
def train(X, y):
    p_spam, p_ham = calculate_prior(y)
    likelihood_spam, likelihood_ham = calculate_likelihood(X, y, p_spam, p_ham)
    return likelihood_spam, likelihood_ham, p_spam, p_ham

In [18]:
def score(y_pred, y):
    return (np.sum(y_pred == y)) / y_pred.shape[0]

In [16]:
X, y = transform_data(df)
likelihood_spam, likelihood_ham, p_spam, p_ham = train(X, y)
y_pred = predict(X, likelihood_spam, likelihood_ham, p_spam, p_ham)

In [19]:
score(y_pred, y)

0.8677315147164394